Code for downloading land use layers and subsequent reclassification of land use codes.
The download and classification of land use was done using [GEE](https://geemap.org/)
This reclassification is based on the values in this [table](https://github.com/kampax/EarthCul/blob/main/Landscape%20variables/LULC_CORINE_Codes.csv)

In [3]:
import ee
import geemap

# Initialize and authenticate Earth Engine
#Remove comment if authentication is required in Google Earth Engine
# ee.Authenticate()
ee.Initialize()

In [6]:
# Load the Features collection
BBs = ee.FeatureCollection("users/cnav/EarthCul/BBs_EarthCul")
#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide

#Filter collection by park
park = "Aiguestortes"
AOI = BBs.filter(ee.Filter.eq("PN", park))
AOI = AOI.geometry().buffer(10000)###Include a 10 km buffer around my area of interest

# Load the Corine Land Cover land use layer
clc = ee.Image("COPERNICUS/CORINE/V20/100m/2018")

# Recategorize CLC values
oldValues = [111,112,121,122,123,124,131,132,133,142,
141,
211,212,213,221,222,223,231,241,242,243,244,
311,312,313,321,322,323,324,331,332,333,334,335,
411,412,421,422,423,511,512,521,522,523
             ]

newValues = [100,100,100,100,100,100,100,100,100,100, #Artificial surfaces 
141,
211,212,213,221,222,223,231,241,242,243,244,
311,312,313,321,322,323,324,331,332,333,334,335,
411,412,421,422,423,511,512,521,522,523
             ]

# Remap raster values
clc_recat = clc.remap(oldValues, newValues, None).rename('landcover')

# Agregar la capa recategorizada al mapa
palette = [
'#ffa6ff', '#ffffa8', '#ffff00', '#e6e600', '#e68000', '#f2a64d', '#e6a600', '#e6e64d', 
'#ffe6a6', '#ffe64d', '#e6cc4d', '#f2cca6', '#80ff00', '#00a600', '#4dff00', '#ccf24d', 
'#a6ff80', '#a6e64d', '#a6f200', '#e6e6e6', '#cccccc', '#ccffcc', '#000000', '#a6e6cc', 
'#a6a6ff', '#4d4dff', '#ccccff', '#e6e6ff', '#a6a6e6', '#00ccf2', '#80f2e6', '#00ffa6', 
'#a6ffe6', '#e6f2ff'

]

# Add the recategorized layer to the map
Map = geemap.Map(center=[40, -100], zoom=4)
Map.addLayer(clc_recat, {'min': 100, 'max': 523, 'palette': palette}, "LandCover recategorized")
Map.addLayer(clc, {}, "Original LandCover")

Map.centerObject(AOI, 10)

Map

Map(center=[42.57826728642684, 0.9164188095819685], controls=(WidgetControl(options=['position', 'transparent_…

In [8]:

import os
# Code to download the generated land use layer
## Define the output folder
base_path = os.getcwd()
out_dir = os.path.join(base_path,"../LULC/"+park)
filename = os.path.join(out_dir, 'LULC.tif')

# Clip layer
image = clc_recat.clip(AOI).unmask()

# Export LULC layer
geemap.ee_export_image(
    image, filename=filename, scale=100, region=AOI, file_per_band=False, crs= "EPSG:32630"
)


Generating URL ...
Please wait ...
Data downloaded to c:\Users\carlo\Documents\EarthCul\Landscape variables\LULC\Aiguestortes\LULC.tif


In [9]:
# If also want to download the original land use layer

filename2 = os.path.join(out_dir, 'Original_LULC.tif')

# Clip layer
image2 = clc.clip(AOI).unmask()

#Export original layer
geemap.ee_export_image(
    image2, filename=filename2, scale=100, region=AOI, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\carlo\Documents\EarthCul\Landscape variables\LULC\Aiguestortes\Original_LULC.tif


In [ ]:
# If you want like to download to the drive

# geemap.ee_export_image_to_drive(
#     image, description='LULC', folder='export', region=AOI, scale=100
# )